In [1]:
import numpy as np

# Time Wrapper

In [2]:
import time
from functools import wraps


def timing_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        total_execution_time = 0
        iterations = 100000

        for _ in range(iterations):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            end_time = time.perf_counter()
            total_execution_time += (end_time - start_time)

        average_execution_time = total_execution_time / iterations
        print(
            f'Average execution time over {iterations} iterations: {average_execution_time:.8f} seconds')

        return result

    return wrapper

## Mean|

In [3]:
@timing_decorator
def custom_mean(x):
    return np.sum(x)/x.size


@timing_decorator
def mean(x):
    return np.mean(x)

In [4]:
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [5]:
custom_mean(arr)

Average execution time over 100000 iterations: 0.00000313 seconds


5.5

In [6]:
mean(arr)

Average execution time over 100000 iterations: 0.00000475 seconds


5.5

# Var

In [7]:
def custom_mean(x):
    return np.sum(x)/x.size

In [8]:
@timing_decorator
def custom_var(x):
    mean_x = custom_mean(x)
    return custom_mean(x**2) - mean_x**2


@timing_decorator
def custom_var_2(x):
    return custom_mean(x**2) - custom_mean(x)**2


@timing_decorator
def custom_var_3(x):
    return custom_mean(np.square(x-custom_mean(x)))


@timing_decorator
def var(x):
    return np.var(x)

In [9]:
@timing_decorator
def custom_var_optimized(x):
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [10]:
@timing_decorator
def custom_unbias_var_optimized(x):
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size-1
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [11]:
custom_unbias_var_optimized(arr)

Average execution time over 100000 iterations: 0.00000680 seconds


5.432098765432102

In [12]:
custom_var(arr)

Average execution time over 100000 iterations: 0.00000688 seconds


8.25

In [13]:
custom_var_optimized(arr)

Average execution time over 100000 iterations: 0.00000665 seconds


8.25

In [14]:
custom_var_2(arr)

Average execution time over 100000 iterations: 0.00000674 seconds


8.25

In [15]:
custom_var_3(arr)

Average execution time over 100000 iterations: 0.00000797 seconds


8.25

In [16]:
var(arr)

Average execution time over 100000 iterations: 0.00001610 seconds


8.25

# Std

In [17]:
def custom_var_optimized(x):
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size-1
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [18]:
@timing_decorator
def custom_std(x):
    return np.sqrt(custom_var_optimized(x))


@timing_decorator
def std(x):
    return np.std(x)

In [19]:
std(arr)

Average execution time over 100000 iterations: 0.00001762 seconds


2.8722813232690143

In [20]:
custom_std(arr)

Average execution time over 100000 iterations: 0.00000753 seconds


2.330686329267004

# Median

In [21]:
@timing_decorator
def custom_median(x):
    x_sort = np.sort(x)
    n = x.size
    if n%2 == 1:
        return x_sort[n//2]
    else:
        m1 = x_sort[n//2]
        m2 = x_sort[n//2-1]
        return (m1+m2)/2

In [22]:
@timing_decorator
def median(x):
    return np.median(x)

In [23]:
median(arr)

Average execution time over 100000 iterations: 0.00001073 seconds


5.5

In [24]:
custom_median(arr)

Average execution time over 100000 iterations: 0.00000151 seconds


5.5

# Mode

In [25]:
@timing_decorator
def custom_mode(x):

    assert len(x.shape) == 1

    counts = np.bincount(x)

    mode_value = np.argmax(counts)

    return mode_value

In [26]:
from scipy import stats

@timing_decorator
def mode(x):
    return stats.mode(x)

In [27]:
mode(arr)

Average execution time over 100000 iterations: 0.00019912 seconds


ModeResult(mode=1, count=1)

In [28]:
custom_mode(arr)

Average execution time over 100000 iterations: 0.00000230 seconds


1

# Percentile

In [29]:
# @timing_decorator
def custom_percentile(x, percentile):
    # Step 1: Sorting the data
    x_sort = np.sort(x)  
    
    # Step 2: Finding the index of the percentile value
    index = (percentile / 100) * (x_sort.size - 1)  
    
    # Step 3: Interpolating to get the percentile value
    lower = np.floor(index).astype(int)  # Finding the lower index
    upper = np.ceil(index).astype(int)   # Finding the upper index
    if lower == upper:  # If index is an integer
        return x_sort[lower]  # Return the value at index
    else:
        # Interpolate between the values at the lower and upper indices
        return x_sort[lower] + (x_sort[upper] - x_sort[lower]) * (index - lower)


In [30]:
@timing_decorator
def percentile(x, percentile):
    return np.percentile(x, percentile)

In [31]:
custom_percentile(arr, percentile=20)

2.8

In [32]:
percentile(arr, percentile=20)

Average execution time over 100000 iterations: 0.00006250 seconds


2.8

# Covariance 

In [33]:
arr1 = np.array([1, 2, 3, 4, 5])
arr2 = np.array([6, 7, 8, 9, 10])

In [34]:
def custom_mean(x):
    return np.sum(x) / x.size

def custom_var(x):
    mean_x = custom_mean(x)
    return np.sum((x - mean_x)**2) / (x.size - 1)
@timing_decorator
def custom_cov(x, y):
    mean_x = custom_mean(x)
    mean_y = custom_mean(y)
    cov_xy = np.sum((x - mean_x) * (y - mean_y)) / (x.size - 1)
    var_x = custom_var(x)
    var_y = custom_var(y)
    cov_matrix = np.array([[var_x, cov_xy], [cov_xy, var_y]])
    return cov_matrix

In [35]:
@timing_decorator
def cov(x,y):
    return np.cov(x ,y)

In [36]:
custom_cov(arr1, arr2)

Average execution time over 100000 iterations: 0.00002695 seconds


array([[2.5, 2.5],
       [2.5, 2.5]])

In [37]:
cov(arr1, arr2)

Average execution time over 100000 iterations: 0.00003306 seconds


array([[2.5, 2.5],
       [2.5, 2.5]])